[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [10]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [23]:
# Read dataset with extra params sep='\t', encoding="latin-1"
dataset = pd.read_csv("data/labeledTrainData.tsv", sep='\t', encoding="latin-1")
dataset.sample(10)

,id,sentiment,review
13851,401_10,1,I just came back from Hong Kong on my summer v...
8094,9668_9,1,"This is a small film , few characters ,theatri..."
21685,7876_8,1,I did not like Chandni Bar from the same direc...
3546,2530_8,1,One of the joys of picking up the recent Bela ...
8164,4047_1,0,"*****WARNING, MAY CONTAIN SPOILERS WHICH WILL ..."
23082,7624_7,1,A pretty memorable movie of the animals-killin...
18443,5501_1,0,I thought that My Favorite Martian was very bo...
11214,4681_7,1,"Overall, I enjoyed this film and would recomme..."
1029,1938_9,1,"The big bad swim has a low budget, indie feel ..."
5970,2717_1,0,Do we really need any more narcissistic garbag...


## 2. Preprocessing

In [18]:
# stop words

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hiep.pham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    # Your code here
    emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W+', '', text.lower())+' '+' '.join(emoticons).replace('-', ''))
    return text


In [20]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] # Your code here


In [21]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = dataset['review']
y = dataset['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [24]:
# Using Pipeline to concat tfidf step and LogisticRegression step
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

error: unterminated character set at position 0

## 4. Evaluate Model

In [25]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

prediction = clf.predict(X_test)
matrix = confusion_matrix(y_test, prediction)
report = classification_report(y_test, prediction)
print (matrix)
print (report)


NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

## 5. Export Model 

In [26]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open(os.path.join('data', '../processedReviewSentiment.pkl'), 'wb'), protocol=4)